# Iterator workflow for cables
To be able to use the Iterator, the following steps need to be followed to install py-ofx on your laptop:
1. Clone the py-ofx repository to your laptop: https://github.com/VanOord/py-ofx;
2. Check out the temp-mixin-fix branch;
3. Install py-ofx by running `pip install -e .` in the top level of the repository.

After following those steps, you should be able to perform the following imports:

In [ ]:
import OrcFxAPI as ofx
from py_ofx.utils import Iterator

The general procedure for working with the Iterator object is as follows:
0. Create example model (for this session only);
1. Prepare your model;
2. Load model file into Iterator;
3. Load checker functions;
4. Define iteration parameter;
5. Run.

## 0. Create example model
For this session we create a model using python so that no *dat file needs to be loaded.

In [ ]:
# Create model
model = ofx.Model()

# Add line object
line = model.CreateObject(ofx.otLine, name="Cable")

## 1. Prepare your model
In principle this is done in the same way as in the previous workflow. The parameters that do not change during the iteration should be all set in advance.

In [ ]:
# Set JONSWAP waves
model.environment.WaveType = "JONSWAP"

# Set wave parameters
model.environment.WaveTp = 8

# Set simulation duration
model.general.StageCount = 2
model.general.StageDuration = [10, 10800]

## 2. Load model into Iterator
Create an Iterator object and load the Orcaflex model into it. For demonstration purposes we set the time interval for checking if the limits are exceeded to 30 seconds (real time).

In [ ]:
it = Iterator(model)
it.limit_check_interval = 30

## 3. Load checker functions
Here the workflow starts deviating significantly from the previous workflow. To let the Iterator object check the limiting criteria, checker functions must be defined and loaded into the object. An example of a simple checker function looks like this:
```
def checker_function(model, period):
    data = model["Cable"].TimeHistory("Effective tension", period, ofx.oeEndA)
    exceeded = data.max() > 42
    return (exceeded, data)
```
The reason this has to be in the form of a function is that the Iterator will change the variable period every time it calls this function so that the checks are done for example at the following periods:
- from start to 0 seconds
- from 0 to 5 seconds
- from 5 to 10 seconds
- from 10 seconds until the end

If the first argument which the checker function returns is True, the simulation will be aborted and restarted after re-setting the iteration parameter.

The second argument that the checker function returns is the data from the TimeHistory or RangeGraph, which can be used for post-processing purposes. This avoids the need to write the same Orcaflex code twice.

Let's use the example checker function above in this model:

In [ ]:
def checker_function(model, period):
    data = model["Cable"].TimeHistory("Effective tension", period, ofx.oeEndA)
    exceeded = data.max() > 42
    return (exceeded, data)

Now the checker function can be loaded into the Iterator:

In [ ]:
it.set_checker_funcs(checker_function)

## 4. Define iteration parameter
Normally we use the significant wave height as iteration parameter. With this function you can set any Orcaflex parameter as iteration parameter. Note that the parameter should be written exactly how Orcaflex defines the parameter, such as WaveHs, WaveHeight etc.

Currently the Iterator can only iterate down, so the step must be negative.

In [ ]:
it.set_iteration_parameter(parameter="WaveHs", start=1.5, end=0, step=-0.25)

## 5. Run
No the Iterator is ready for running. Notice that the simulation gets aborted and output is given with the name of the checker function that caused the exit.

In [ ]:
it.run()

Remember that the failed simulations are being reset and no data of those simulations is saved by default. The failure modes of the failed simulations are stored:

In [ ]:
it.failure_modes

After running, the model is complete and results can be processed

In [ ]:
model.state

In [ ]:
# Reset the model before starting the assignments below
model.Reset()
model.state

### Assignment 1
Write your own checker function for curvature. The maximum allowable curvature is 0.084.

In [ ]:
def check_curvature(model, period):
    # Write your code here
    
    return (exceeded, data)

In [ ]:
# You can test if your checker works the way you expect it to be with statics:
model.CalculateStatics()
check_curvature(model, ofx.pnStaticState)

### Assignment 2
Create a new Iterator object and load the existing model into it. For this assignment the check interval is set to 30 sec.

In [ ]:
it = 
it.limit_check_interval = 30

### Assignment 3
Add your checker function to the iterator object.

### Assignment 4
Define the iteration parameter Hs and start at 1.5m.

## Prerunners
Sometimes when running an iteration, other parameters besides the iteration parameter need to be changed for each iteration step. This can be done using a prerunner. A few standard prerunners are included in py-ofx:
- seed selection
- wave gamma selection

These can easily be imported and added to the iterator. Do note that the order in which the prerunners are added is also the order in which the Iterator will call the prerunners. <b>As the peak enhancement factor has an influence on waves in the seed, set_gamma should be added before set_seed</b>.

In [ ]:
from py_ofx.utils import set_gamma, set_seed

In [ ]:
it.add_prerunner(set_gamma)
it.add_prerunner(set_seed)

### Assignment 5
Run your model :)

# Documentation
If you want to know more about the Iterator, you can find a lot of documentation in the scripts.

In [ ]:
help(Iterator)